In [1]:
import sys
sys.path.append("../")


import os
%matplotlib inline
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from patsy import dmatrices
from openpyxl import load_workbook
import engarde.decorators as ed

from library import test_data
@ed.verify(test_data.allyearsandtpsd)
@ed.verify(test_data.alldois)
def load(data):
    return data

In [9]:
data_path = '/Users/kylieleblancKylie/domino/dofis/data/'
table_path = '/Users/kylieleblancKylie/domino/dofis/results/impact/'
data = pd.read_csv(os.path.join(data_path, 'clean', 'cits.csv'),
                  sep=",", low_memory= False)
#load(data)
data.tail()
data[data.doi == True].district.nunique()

813

In [3]:
data[data.district == 250907][['district','distname','year','year_centered', 'avescores','doi', 'treat']]

,district,distname,year,year_centered,avescores,doi,treat
1020,250907,WINNSBORO ISD,2012,-6.0,0.375915,True,1
2046,250907,WINNSBORO ISD,2013,-5.0,0.629580,True,1
3071,250907,WINNSBORO ISD,2014,-4.0,0.764338,True,1
4095,250907,WINNSBORO ISD,2015,-3.0,0.592911,True,1
5119,250907,WINNSBORO ISD,2016,-2.0,0.719108,True,1
6142,250907,WINNSBORO ISD,2017,-1.0,0.838382,True,1
7165,250907,WINNSBORO ISD,2018,0.0,0.820199,True,1


In [4]:
def coef_with_stars(coef, pvalue):
    if pvalue >.05:
        coef = str(coef)
    if pvalue <= .05:
        coef = str(coef) + '*'
    if pvalue <= .01:
        coef = coef + '*'
    if pvalue <= .001:
        coef = coef + '*'
    return(coef)
test = coef_with_stars(.1, .005)

In [5]:
data.district.nunique()

1029

In [6]:
data[pd.notnull(data.year_centered)].district.nunique()

981

# Drop districts with missing doi_year and missing pscores

In [7]:
reg_vars = ['year_centered', 'avescores', 'treat', 'psweight1', 'psweight2']
data = data.dropna(subset=reg_vars)
data.district.nunique()

971

# No Matching

In [8]:
y, X = dmatrices('avescores ~  treat + year_centered + treatyear + yearpost1 + treatpostyear1 + yearpost2 + treatpostyear2 ',
                 data=data, return_type='dataframe', NA_action = "drop")
cluster = data.dropna(subset=reg_vars).district
mod = smf.OLS(y, X)    # Describe model
res = mod.fit(cov_type = 'cluster',  cov_kwds={'groups': pd.DataFrame(cluster)})       # Fit model
print(res.summary())   # Summarize model
row_1 = []
for coef in ['treat', 'year_centered', 'treatyear', 'yearpost1', 'treatpostyear1']:
    row_1.append(coef_with_stars(res.params[coef].round(2), res.pvalues[coef]))
    row_1.append('(' + str(res.bse[coef].round(2)) + ')')
row_1.append('')
row_1.append('')
row_1.append(res.rsquared_adj.round(2))
row_1

                            OLS Regression Results                            
Dep. Variable:              avescores   R-squared:                       0.111
Model:                            OLS   Adj. R-squared:                  0.110
Method:                 Least Squares   F-statistic:                     124.0
Date:                Thu, 18 Jul 2019   Prob (F-statistic):          5.28e-130
Time:                        13:39:22   Log-Likelihood:                -7138.0
No. Observations:                6793   AIC:                         1.429e+04
Df Residuals:                    6785   BIC:                         1.435e+04
Df Model:                           7                                         
Covariance Type:              cluster                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -0.1198      0.049     -2.

['0.48***',
 '(0.06)',
 '0.09***',
 '(0.01)',
 '-0.0',
 '(0.01)',
 '-0.06*',
 '(0.03)',
 '0.03*',
 '(0.01)',
 '',
 '',
 0.11]

# Matching without pre-tests


In [9]:
y, X = dmatrices('avescores ~  treat + year_centered + treatyear + yearpost1 + treatpostyear1 + yearpost2 + treatpostyear2 + yearpost3 + treatpostyear3',
                 data=data, return_type='dataframe', NA_action = "drop")
cluster = data.dropna(subset=reg_vars).district
psweights = data.dropna(subset=reg_vars).psweight1
mod = smf.WLS(y, X, weights = psweights)    # Describe model
res = mod.fit(cov_type = 'cluster',  cov_kwds={'groups': pd.DataFrame(cluster)})       # Fit model
print(res.summary())   # Summarize model
row_2 = []
for coef in ['treat', 'year_centered', 'treatyear', 'yearpost1', 'treatpostyear1']:
    row_2.append(coef_with_stars(res.params[coef].round(2), res.pvalues[coef]))
    row_2.append('(' + str(res.bse[coef].round(2)) + ')')
row_2.append('X')
row_2.append('')
row_2.append(res.rsquared_adj.round(2))
row_2

                            WLS Regression Results                            
Dep. Variable:              avescores   R-squared:                       0.087
Model:                            WLS   Adj. R-squared:                  0.086
Method:                 Least Squares   F-statistic:                     102.5
Date:                Thu, 18 Jul 2019   Prob (F-statistic):          4.09e-112
Time:                        13:39:22   Log-Likelihood:                -8001.2
No. Observations:                6793   AIC:                         1.602e+04
Df Residuals:                    6785   BIC:                         1.607e+04
Df Model:                           7                                         
Covariance Type:              cluster                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.0083      0.057      0.

/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1633: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/statsmodels/base/model.py:1532: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 9, but rank is 7
  'rank is %d' % (J, J_), ValueWarning)
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/statsmodels/base/model.py:1100: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWar

['0.32***',
 '(0.06)',
 '0.08***',
 '(0.01)',
 '0.0',
 '(0.01)',
 '-0.07*',
 '(0.03)',
 '0.03*',
 '(0.01)',
 'X',
 '',
 0.09]

# Matching with Pre-tests

In [10]:
data = data.dropna(subset = ['avescores', 'psweight2', 'scores15', 'scores14', 'scores13', 'scores12'], how = 'any')
y, X = dmatrices('avescores ~  treat + year_centered + treatyear + yearpost1 + treatpostyear1 + yearpost2 + treatpostyear2 + yearpost3 + treatpostyear3',
                 data=data, return_type='dataframe', NA_action = "drop")
cluster = data[pd.notnull(data.scores14)].distname
psweights = data[pd.notnull(data.scores14)].psweight2
mod = smf.WLS(y, X, weights = psweights)    # Describe model
res = mod.fit(cov_type = 'cluster',  cov_kwds={'groups': pd.DataFrame(cluster)})       # Fit model
print(res.summary())   # Summarize model
row_3 = []
for coef in ['treat', 'year_centered', 'treatyear', 'yearpost1', 'treatpostyear1']:
    row_3.append(coef_with_stars(res.params[coef].round(2), res.pvalues[coef]))
    row_3.append('(' + str(res.bse[coef].round(2)) + ')')
row_3.append('X')
row_3.append('X')
row_3.append(res.rsquared_adj.round(2))
row_3

                            WLS Regression Results                            
Dep. Variable:              avescores   R-squared:                       0.064
Model:                            WLS   Adj. R-squared:                  0.064
Method:                 Least Squares   F-statistic:                     92.47
Date:                Thu, 18 Jul 2019   Prob (F-statistic):          2.85e-103
Time:                        13:39:22   Log-Likelihood:                -8021.0
No. Observations:                6793   AIC:                         1.606e+04
Df Residuals:                    6785   BIC:                         1.611e+04
Df Model:                           7                                         
Covariance Type:              cluster                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.1660      0.057      2.

/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1633: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/statsmodels/base/model.py:1532: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 9, but rank is 7
  'rank is %d' % (J, J_), ValueWarning)
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/statsmodels/base/model.py:1100: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWar

['0.13*',
 '(0.06)',
 '0.08***',
 '(0.01)',
 '0.0',
 '(0.01)',
 '-0.06*',
 '(0.03)',
 '0.04**',
 '(0.01)',
 'X',
 'X',
 0.06]

In [11]:
df = pd.DataFrame(
    {'Model 1': row_1,
    'Model 2': row_2,
    'Model 3': row_3})
df

,Model 1,Model 2,Model 3
0,0.48***,0.32***,0.13*
1,(0.06),(0.06),(0.06)
2,0.09***,0.08***,0.08***
3,(0.01),(0.01),(0.01)
4,-0.0,0.0,0.0
5,(0.01),(0.01),(0.01)
6,-0.06*,-0.07*,-0.06*
7,(0.03),(0.03),(0.03)
8,0.03*,0.03*,0.04**
9,(0.01),(0.01),(0.01)


In [12]:
df = pd.DataFrame(
    {'Model 1': row_1,
    'Model 2': row_2,
    'Model 3': row_3})
df

,Model 1,Model 2,Model 3
0,0.48***,0.32***,0.13*
1,(0.06),(0.06),(0.06)
2,0.09***,0.08***,0.08***
3,(0.01),(0.01),(0.01)
4,-0.0,0.0,0.0
5,(0.01),(0.01),(0.01)
6,-0.06*,-0.07*,-0.06*
7,(0.03),(0.03),(0.03)
8,0.03*,0.03*,0.04**
9,(0.01),(0.01),(0.01)


In [13]:
file = table_path + 'results_main.xlsx'
wb = load_workbook(file)
ws = wb.active
df
for model, col in zip(['Model 1', 'Model 2', 'Model 3'], [2,3,4]):
    row_n = 3
    for ob in df[model]:
        ws.cell(row=row_n, column=col).value = ob
        row_n = row_n + 1
wb.save(file)

# Fixed Effects

In [14]:
data_fe = pd.get_dummies(data = data, columns = ['year'])
reg_vars = ['year_centered', 'avescores', 'treat', 'psweight1', 'psweight2']
data_fe = data_fe.dropna(subset=reg_vars)
#data_fe = data_fe.dropna(reg_vars)
data_fe.sample(10)

,Unnamed: 0,Unnamed: 0.1,district,distname,cntyname,distischarter,rating_academic,rating_financial,eligible,type,...,ps2,psweight2,pswtshare2,year_2012,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018
4764,4764,5702,161909,MCGREGOR ISD,MCLENNAN,N,M,Pass,True,G,...,0.807854,1.237847,0.121673,0,0,0,0,1,0,0
4054,4054,4850,243901,BURKBURNETT ISD,WICHITA,N,M,Pass,True,D,...,0.905167,1.104769,0.108592,0,0,0,1,0,0,0
272,272,346,69901,ROCKSPRINGS ISD,EDWARDS,N,NaN,NaN,NaN,H,...,0.568267,1.759738,0.172971,1,0,0,0,0,0,0
3811,3811,4568,177901,ROSCOE COLLEGIATE ISD,NOLAN,N,M,Pass,True,F,...,0.715484,1.397656,0.137381,0,0,0,1,0,0,0
4584,4584,5513,111902,LIPAN ISD,HOOD,N,M,Pass,True,F,...,0.914745,1.093201,0.107455,0,0,0,0,1,0,0
42,42,48,14905,HOLLAND ISD,BELL,N,NaN,NaN,NaN,H,...,0.867745,7.561130,0.739792,1,0,0,0,0,0,0
1753,1753,2107,175903,CORSICANA ISD,NAVARRO,N,M,NaN,NaN,D,...,0.810445,1.233890,0.121284,0,1,0,0,0,0,0
4896,4896,5842,189902,PRESIDIO ISD,PRESIDIO,N,M,Pass,True,G,...,0.591166,1.691571,0.166271,0,0,0,0,1,0,0
428,428,559,102903,WASKOM ISD,HARRISON,N,NaN,NaN,NaN,G,...,0.864957,1.156127,0.113640,1,0,0,0,0,0,0
1343,1343,1626,76903,ROBY ISD,FISHER,N,M,NaN,NaN,H,...,0.838630,1.192421,0.117207,0,1,0,0,0,0,0


In [15]:
y, X = dmatrices('avescores ~  + doi + treatpost +  year_2013 + year_2014 + year_2015 + year_2016 + year_2017 + year_2018',
                 data=data_fe, return_type='dataframe', NA_action = "drop")
cluster = data_fe.dropna(subset=reg_vars).district
mod = smf.OLS(y, X)    # Describe model
res = mod.fit(cov_type = 'cluster',  cov_kwds={'groups': pd.DataFrame(cluster)})       # Fit model
print(res.summary())   # Summarize model


                            OLS Regression Results                            
Dep. Variable:              avescores   R-squared:                       0.109
Model:                            OLS   Adj. R-squared:                  0.108
Method:                 Least Squares   F-statistic:                     268.6
Date:                Thu, 18 Jul 2019   Prob (F-statistic):          6.59e-240
Time:                        13:39:23   Log-Likelihood:                -7147.1
No. Observations:                6793   AIC:                         1.431e+04
Df Residuals:                    6784   BIC:                         1.437e+04
Df Model:                           8                                         
Covariance Type:              cluster                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -0.4596      0.046     -9.989      